# Run VK1

**Download Kaggle version**

Spectrograms of F3 and F4 in two channels.

- Definitions for scoring.
- Training run.
- Scoring.

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import regularizers
import matplotlib.pyplot as plt

base_dir = '../../kaggle_data/hms'
# base_dir = '../../data/hms'
# base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'

devset_dir = '../data'

# path_train = f'{devset_dir}/03_single_spectrograms_v1_train.npy'
# path_train_items = f'{devset_dir}/03_single_spectrograms_v1_train_items.npy'
# path_val = f'{devset_dir}/03_single_spectrograms_v1_val.npy'
# path_val_items = f'{devset_dir}/03_single_spectrograms_v1_val_items.npy'
# path_test = f'{devset_dir}/03_single_spectrograms_v1_test.npy'
# path_test_items = f'{devset_dir}/03_single_spectrograms_v1_test_items.npy'

path_train = f'{devset_dir}/03_single_spectrograms_reduced_v1_train.npy'
path_train_items = f'{devset_dir}/03_single_spectrograms_reduced_v1_train_items.npy'
path_val = f'{devset_dir}/03_single_spectrograms_reduced_v1_val.npy'
path_val_items = f'{devset_dir}/03_single_spectrograms_reduced_v1_val_items.npy'
path_test = f'{devset_dir}/03_single_spectrograms_reduced_v1_test.npy'
path_test_items = f'{devset_dir}/03_single_spectrograms_reduced_v1_test_items.npy'


## Definitions

In [7]:
import pandas.api.types
from typing import Union
from typing import Optional


class ParticipantVisibleError(Exception):
    pass


class HostVisibleError(Exception):
    pass


def treat_as_participant_error(error_message: str, solution: Union[pd.DataFrame, np.ndarray]) -> bool:
    ''' Many metrics can raise more errors than can be handled manually. This function attempts
    to identify errors that can be treated as ParticipantVisibleError without leaking any competition data.

    If the solution is purely numeric, and there are no numbers in the error message,
    then the error message is sufficiently unlikely to leak usable data and can be shown to participants.

    We expect this filter to reject many safe messages. It's intended only to reduce the number of errors we need to manage manually.
    '''
    # This check treats bools as numeric
    if isinstance(solution, pd.DataFrame):
        solution_is_all_numeric = all([pandas.api.types.is_numeric_dtype(x) for x in solution.dtypes.values])
        solution_has_bools = any([pandas.api.types.is_bool_dtype(x) for x in solution.dtypes.values])
    elif isinstance(solution, np.ndarray):
        solution_is_all_numeric = pandas.api.types.is_numeric_dtype(solution)
        solution_has_bools = pandas.api.types.is_bool_dtype(solution)

    if not solution_is_all_numeric:
        return False

    for char in error_message:
        if char.isnumeric():
            return False
    if solution_has_bools:
        if 'true' in error_message.lower() or 'false' in error_message.lower():
            return False
    return True


def safe_call_score(metric_function, solution, submission, **metric_func_kwargs):
    '''
    Call score. If that raises an error and that already been specifically handled, just raise it.
    Otherwise make a conservative attempt to identify potential participant visible errors.
    '''
    try:
        score_result = metric_function(solution, submission, **metric_func_kwargs)
    except Exception as err:
        error_message = str(err)
        if err.__class__.__name__ == 'ParticipantVisibleError':
            raise ParticipantVisibleError(error_message)
        elif err.__class__.__name__ == 'HostVisibleError':
            raise HostVisibleError(error_message)
        else:
            if treat_as_participant_error(error_message, solution):
                raise ParticipantVisibleError(error_message)
            else:
                raise err
    return score_result


def verify_valid_probabilities(df: pd.DataFrame, df_name: str):
    """ Verify that the dataframe contains valid probabilities.

    The dataframe must be limited to the target columns; do not pass in any ID columns.
    """
    if not pandas.api.types.is_numeric_dtype(df.values):
        raise ParticipantVisibleError(f'All target values in {df_name} must be numeric')

    if df.min().min() < 0:
        raise ParticipantVisibleError(f'All target values in {df_name} must be at least zero')

    if df.max().max() > 1:
        raise ParticipantVisibleError(f'All target values in {df_name} must be no greater than one')

    if not np.allclose(df.sum(axis=1), 1):
        raise ParticipantVisibleError(f'Target values in {df_name} do not add to one within all rows')


def kl_divergence(solution: pd.DataFrame, submission: pd.DataFrame, epsilon: float, micro_average: bool, sample_weights: Optional[pd.Series]):
    # Overwrite solution for convenience
    for col in solution.columns:
        # Prevent issue with populating int columns with floats
        if not pandas.api.types.is_float_dtype(solution[col]):
            solution[col] = solution[col].astype(float)

        # Clip both the min and max following Kaggle conventions for related metrics like log loss
        # Clipping the max avoids cases where the loss would be infinite or undefined, clipping the min
        # prevents users from playing games with the 20th decimal place of predictions.
        submission[col] = np.clip(submission[col], epsilon, 1 - epsilon)

        y_nonzero_indices = solution[col] != 0
        solution[col] = solution[col].astype(float)
        solution.loc[y_nonzero_indices, col] = solution.loc[y_nonzero_indices, col] * np.log(solution.loc[y_nonzero_indices, col] / submission.loc[y_nonzero_indices, col])
        # Set the loss equal to zero where y_true equals zero following the scipy convention:
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.rel_entr.html#scipy.special.rel_entr
        solution.loc[~y_nonzero_indices, col] = 0

    if micro_average:
        return np.average(solution.sum(axis=1), weights=sample_weights)
    else:
        return np.average(solution.mean())

def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        row_id_column_name: str,
        epsilon: float=10**-15,
        micro_average: bool=True,
        sample_weights_column_name: Optional[str]=None
    ) -> float:
    ''' The Kullback-Leibler divergence.
    The KL divergence is technically undefined/infinite where the target equals zero.

    This implementation always assigns those cases a score of zero; effectively removing them from consideration.
    The predictions in each row must add to one so any probability assigned to a case where y == 0 reduces
    another prediction where y > 0, so crucially there is an important indirect effect.

    https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence

    solution: pd.DataFrame
    submission: pd.DataFrame
    epsilon: KL divergence is undefined for p=0 or p=1. If epsilon is not null, solution and submission probabilities are clipped to max(eps, min(1 - eps, p).
    row_id_column_name: str
    micro_average: bool. Row-wise average if True, column-wise average if False.

    Examples
    --------
    >>> import pandas as pd
    >>> row_id_column_name = "id"
    >>> score(pd.DataFrame({'id': range(4), 'ham': [0, 1, 1, 0], 'spam': [1, 0, 0, 1]}), pd.DataFrame({'id': range(4), 'ham': [.1, .9, .8, .35], 'spam': [.9, .1, .2, .65]}), row_id_column_name=row_id_column_name)
    0.216161...
    >>> solution = pd.DataFrame({'id': range(3), 'ham': [0, 0.5, 0.5], 'spam': [0.1, 0.5, 0.5], 'other': [0.9, 0, 0]})
    >>> submission = pd.DataFrame({'id': range(3), 'ham': [0, 0.5, 0.5], 'spam': [0.1, 0.5, 0.5], 'other': [0.9, 0, 0]})
    >>> score(solution, submission, 'id')
    0.0
    >>> solution = pd.DataFrame({'id': range(3), 'ham': [0, 0.5, 0.5], 'spam': [0.1, 0.5, 0.5], 'other': [0.9, 0, 0]})
    >>> submission = pd.DataFrame({'id': range(3), 'ham': [0.2, 0.3, 0.5], 'spam': [0.1, 0.5, 0.5], 'other': [0.7, 0.2, 0]})
    >>> score(solution, submission, 'id')
    0.160531...
    '''
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    sample_weights = None
    if sample_weights_column_name:
        if sample_weights_column_name not in solution.columns:
            raise ParticipantVisibleError(f'{sample_weights_column_name} not found in solution columns')
        sample_weights = solution.pop(sample_weights_column_name)

    if sample_weights_column_name and not micro_average:
        raise ParticipantVisibleError('Sample weights are only valid if `micro_average` is `True`')

    for col in solution.columns:
        if col not in submission.columns:
            raise ParticipantVisibleError(f'Missing submission column {col}')

    verify_valid_probabilities(solution, 'solution')
    verify_valid_probabilities(submission, 'submission')

    return safe_call_score(kl_divergence, solution, submission, epsilon=epsilon, micro_average=micro_average, sample_weights=sample_weights)


In [17]:
# train_set = np.load(f'{devset_dir}/03_single_spectrograms_v1_train.npy')
# train_items = np.load(f'{devset_dir}/03_single_spectrograms_v1_train_items.npy')
# val_set = np.load(f'{devset_dir}/03_single_spectrograms_v1_val.npy')
# val_items = np.load(f'{devset_dir}/03_single_spectrograms_v1_val_items.npy')
# test_set = np.load(f'{devset_dir}/03_single_spectrograms_v1_test.npy')
# test_items = np.load(f'{devset_dir}/03_single_spectrograms_v1_test_items.npy')
# print(f'{train_set.shape[0]} observations in training set.')
# print(f'{val_set.shape[0]} observations in validation set.')
# print(f'{test_set.shape[0]} observations in testing set.')

## Train

In [8]:
#
# Data generator using numpy and no pandas.
#
# Spectrograms
# 10 seconds slice
# 2 channels
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        items: [eeg_id, eeg_sub_id, idx of offset, target]
        '''
        self.n_channels = 2
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size), dtype=int)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int16(item[2]), :, :, :]
            # Store class
            y[i] = np.int16(item[3])

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [9]:

def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    #max1 = keras.layers.MaxPooling1D(pool_size=2)(input_layer)
    
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same")(input_layer)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    # conv1 = keras.layers.MaxPooling2D(pool_size=8)(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=7, padding="same")(conv1)
    #conv2 = keras.layers.BatchNormalization()(conv2)
    # conv2 = keras.layers.MaxPooling2D(pool_size=8)(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=256, kernel_size=7, padding="same")(conv2)
    #conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.MaxPooling2D(pool_size=2)(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    conv4 = keras.layers.Conv1D(filters=512, kernel_size=3, padding="same")(conv3)
    conv4 = keras.layers.BatchNormalization()(conv4)
    conv4 = keras.layers.MaxPooling2D(pool_size=4)(conv4)
    conv4 = keras.layers.ReLU()(conv4)

    fltn  = keras.layers.Flatten()(conv4) 
    
    relu1 = keras.layers.Dense(256)(fltn)
    relu1 = keras.layers.ReLU()(relu1)

    relu2 = keras.layers.Dense(64)(relu1)
    relu2 = keras.layers.ReLU(64)(relu2)

#     lin = keras.layers.Dense(2)(relu2)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(relu2)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [10]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 416
Observations in validation set: 128


In [22]:
dim = training_generator.get_dim()

model = make_model(input_shape=dim, num_classes=6)
model.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=tf.keras.metrics.KLDivergence(
                    name='kullback_leibler_divergence', dtype=None
                    )
            )

model.fit(training_generator, epochs=1, validation_data=validation_generator)

13/13 [==============================] - 376s 29s/step - loss: 4.7281 - accuracy: 0.1650 - val_loss: 2.6698 - val_accuracy: 0.1800


## Score

In [23]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

#
# Test Data generator: for predicting
# using own test set.
# (Not for predicting LB)
#

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        items: [eeg_id, eeg_sub_id, idx of offset, target, ...]
        '''
        self.n_channels = 2
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int16(item[2]), :, :, :]

        return X
    
                
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)

y_pred = model.predict(test_generator)
    


4/4 [==============================] - 21s 5s/step


In [24]:
y_pred

array([[1.6577519e-01, 1.6426449e-01, 1.6785835e-01, 1.7222977e-01,
        1.6519608e-01, 1.6467607e-01],
       [3.0143499e-01, 9.7424351e-02, 1.7135055e-01, 1.6934478e-01,
        1.1828900e-01, 1.4215627e-01],
       [1.6569136e-01, 1.6416799e-01, 1.6785404e-01, 1.7209414e-01,
        1.6552234e-01, 1.6467015e-01],
       [1.6553679e-01, 1.6415414e-01, 1.6778225e-01, 1.7213893e-01,
        1.6567042e-01, 1.6471750e-01],
       [1.6557640e-01, 1.6422121e-01, 1.6775626e-01, 1.7231408e-01,
        1.6539159e-01, 1.6474041e-01],
       [1.6575530e-01, 1.6420758e-01, 1.6783547e-01, 1.7209406e-01,
        1.6541541e-01, 1.6469210e-01],
       [1.6557972e-01, 1.6419984e-01, 1.6778640e-01, 1.7214333e-01,
        1.6555798e-01, 1.6473272e-01],
       [1.6554473e-01, 1.6417181e-01, 1.6780341e-01, 1.7211093e-01,
        1.6565093e-01, 1.6471815e-01],
       [1.7138892e-01, 1.5923227e-01, 1.6824876e-01, 1.7618155e-01,
        1.6267794e-01, 1.6227053e-01],
       [1.6558611e-01, 1.6426310e-01,

In [25]:
# test_items = np.load(f'{devset_dir}/03_single_spectrograms_v1_test_items.npy')
test_items = np.load(f'{devset_dir}/03_single_spectrograms_reduced_v1_test_items.npy')
test_items

array([[2.91959911e+09, 9.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.38832815e+09, 4.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.33388099e+09, 0.00000000e+00, 2.00000000e+00, 5.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00],
       [2.42874105e+08, 1.00000000e+00, 3.00000000e+00, 3.00000000e+00,
        0.00000000e+00, 2.30769231e-01, 0.00000000e+00, 3.84615385e-01,
        0.00000000e+00, 3.84615385e-01],
       [2.56055520e+09, 0.00000000e+00, 4.00000000e+00, 5.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00],
       [2.37763612e+09, 5.00000000e+00, 5.00000000e+00, 1.00000000e+00,
   

In [39]:
df_test_items = pd.DataFrame(test_items)
df_test_items[0] = df_test_items[0].astype(int)
df_test_items

,0,1,2,3,4,5,6,7,8,9
0,2919599106,9.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1388328149,4.0,1.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,2333880986,0.0,2.0,5.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
3,242874105,1.0,3.0,3.0,0.0,0.230769,0.000000,0.384615,0.000000,0.384615
4,2560555204,0.0,4.0,5.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
95,3015725284,4.0,95.0,2.0,0.0,0.000000,0.333333,0.000000,0.333333,0.333333
96,736099120,0.0,96.0,5.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
97,3816121998,0.0,97.0,1.0,0.0,0.333333,0.000000,0.333333,0.000000,0.333333
98,2676914434,19.0,98.0,3.0,0.0,0.000000,0.000000,0.666667,0.333333,0.000000


In [40]:

sub = pd.DataFrame({'eeg_id':df_test_items[0]})
sub[TARGETS] = np.round(y_pred,6)
sub.to_csv('submission.csv',index=False)
sub


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,2919599106,0.165775,0.164264,0.167858,0.172230,0.165196,0.164676
1,1388328149,0.301435,0.097424,0.171351,0.169345,0.118289,0.142156
2,2333880986,0.165691,0.164168,0.167854,0.172094,0.165522,0.164670
3,242874105,0.165537,0.164154,0.167782,0.172139,0.165670,0.164718
4,2560555204,0.165576,0.164221,0.167756,0.172314,0.165392,0.164740
...,...,...,...,...,...,...,...
95,3015725284,0.165535,0.164122,0.167803,0.172173,0.165669,0.164698
96,736099120,0.165511,0.164382,0.167886,0.172315,0.164602,0.165305
97,3816121998,0.167423,0.164334,0.167089,0.172428,0.163189,0.165537
98,2676914434,0.165589,0.164178,0.167829,0.172097,0.165636,0.164671


In [41]:
df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
df_test_scoring.columns = sub.columns
df_test_scoring

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,2919599106,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1388328149,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,2333880986,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
3,242874105,0.0,0.230769,0.000000,0.384615,0.000000,0.384615
4,2560555204,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...
95,3015725284,0.0,0.000000,0.333333,0.000000,0.333333,0.333333
96,736099120,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
97,3816121998,0.0,0.333333,0.000000,0.333333,0.000000,0.333333
98,2676914434,0.0,0.000000,0.000000,0.666667,0.333333,0.000000


In [42]:
score(df_test_scoring, sub, 'eeg_id')

/tmp/ipykernel_54191/1033736718.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_54191/1033736718.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_54191/1033736718.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

2.2980015586907085